In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
from random import randint
im_size = 120
batch_size = 32


num_samples = 10222

num_class = 120
steps_per_epoch = num_samples//batch_size

print(steps_per_epoch)
epochs = 2

def data_gen(batch_size, image_size):
    
    df_train = pd.read_csv('../input/labels.csv')
    
    targets_series = pd.Series(df_train['breed'])
    one_hot = pd.get_dummies(targets_series, sparse = True)
    one_hot_labels = np.asarray(one_hot)


    
    fn = pd.Series(df_train['id'])

    
    x_train = []
    y_train = []
  
   
    while True:
        for i in range(batch_size):
            #index = np.random.choice(fn.shape[0],1)
            index = randint(0,fn.shape[0]-1)
            img = cv2.imread('../input/train/{}.jpg'.format(fn[index]))
            x_train.append(cv2.resize(img,(image_size,image_size)))
            label = one_hot_labels[index]
            y_train.append(label)
        y_train_raw = np.array(y_train, np.uint8)
        x_train_raw = np.array(x_train, np.float32) / 255.
        print(i)
        yield x_train_raw, y_train_raw


319


Test data processing:

In [3]:
# Create the base pre-trained model
# Can't download weights in the kernel
base_model = MobileNet(#weights='imagenet',
    weights = 'imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Add a new top layer
x = base_model.output
x = Flatten()(x)
predictions = Dense(num_class, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

ValueError: When setting`include_top=True`, `input_shape` should be (224, 224, 3).

In [ ]:
model.fit_generator(data_gen( batch_size=batch_size, image_size = im_size),
                    steps_per_epoch=2,
                    epochs=2, verbose =1)


Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [ ]:
x_test = []
    

df_test = pd.read_csv('../input/sample_submission.csv')

for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
 
x_test  = np.array(x_test, np.float32) / 255.


In [ ]:
preds = model.predict(x_test, verbose=1)

In [ ]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', df_test['id'])
sub.head(10358)
df_test.to_csv('pred_vgg.csv', index=None)